In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

from pymatgen.core.structure import Lattice, Structure
from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer


In [3]:
class PerovskiteRotater():
    def __init__(self, angels: list, antiphases: list):
        self.angels = angels
        self.antiphases = antiphases
        self.ORIGIN_COORD = np.array([[0.5, 0, 0], [0, 0.5, 0], [0, 0, 0.5]])
        self.coords = [self.ORIGIN_COORD.copy() for _ in range(8)]
        self.rotated_coords = np.zeros((8, 3, 3))
        self.rotate_rules = self._decide_rotate_rules()

    def _decide_rotate_rules(self):
        rotate_rules = np.array([
            [1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1],
            [-1, -1, 1], [-1, 1, -1], [1, -1, -1], [1, 1, 1]
        ])
        for i in range(8):
            bin_i = bin(i)[2:].zfill(3)[::-1]
            for j, bit in enumerate(bin_i):
                # a, minus
                if j == 0 and bit == '1' and self.antiphases[0] == True:
                    rotate_rules[i][j] *= -1
                # b, minus
                if j == 1 and bit == '1' and self.antiphases[1] == True:
                    rotate_rules[i][j] *= -1
                # c, minus
                if j == 2 and bit == '1' and self.antiphases[2] == True:
                    rotate_rules[i][j] *= -1
        return rotate_rules

    def _rotate_unit(self, rotate_rule, coord):
        rotater = R.from_euler('XYZ', rotate_rule*self.angels, degrees=True)
        return rotater.apply(coord)

    def _merge_coords(self, coords):
        rep_coords = np.full((8, 3, 3), 0.5)
        for i in range(8):
            bin_i = bin(i)[2:].zfill(3)[::-1]
            shift = np.array([int(bin_i[0]), int(bin_i[1]), int(bin_i[2])])
            rep_coords[i] = rep_coords[i] + coords[i] + shift
        return np.concatenate([rep_coords[j] for j in range(8)])

    def rotate(self):
        for i in range(8):
            rotate_rule = self.rotate_rules[i]
            coord = self.coords[i]
            self.rotated_coords[i] = self._rotate_unit(rotate_rule, coord)
        return self._merge_coords(self.rotated_coords)


def get_abx1x2_rotated_structure(pr: PerovskiteRotater, elements: list) -> Structure:
    lattice = Lattice.from_parameters(a=4, b=4, c=4,
                                      alpha=90, beta=90, gamma=90)
    A_B_SITE_COORD = np.array([
        # A site
        [0, 0, 0], [0, 2, 0], [2, 0, 0], [2, 2, 0],
        [0, 0, 2], [2, 0, 2], [0, 2, 2], [2, 2, 2],
        # B site
        [1, 1, 1], [3, 1, 1], [1, 3, 1], [3, 3, 1],
        [1, 1, 3], [1, 3, 3], [3, 1, 3], [3, 3, 3],
    ])
    coords = np.concatenate([A_B_SITE_COORD, pr.rotate()*2])
    a, b, x1, x2 = elements
    species = [a] * 8 + [b] * 8 + [x2, x2, x1] * 8
    return Structure(lattice=lattice, species=species,
                     coords=coords, coords_are_cartesian=True)


In [4]:
import unittest


class TestPerovskiteRotater(unittest.TestCase):
    def setUp(self):
        self.elements = ['Ba', 'Ti', 'O', 'O']

    def test_a0_a0_a0(self):
        pr = PerovskiteRotater(angels=[0, 0, 0], antiphases=[
                               False, False, False])
        s = get_abx1x2_rotated_structure(pr, self.elements)
        
        expected = 'Pm-3m'  # 221
        actual = SpacegroupAnalyzer(s).get_space_group_symbol()
        self.assertEqual(expected, actual)

    def test_am_am_am(self):
        pr = PerovskiteRotater(angels=[1, 1, 1], antiphases=[True, True, True])
        s = get_abx1x2_rotated_structure(pr, self.elements)
        expected = 'R-3c'  # 167
        actual = SpacegroupAnalyzer(s).get_space_group_symbol()
        self.assertEqual(expected, actual)

    def test_ap_bp_cp(self):
        pr = PerovskiteRotater(angels=[1, 2, 3], antiphases=[
                               False, False, False])
        s = get_abx1x2_rotated_structure(pr, self.elements)
        expected = 'Immm'  # 71
        actual = SpacegroupAnalyzer(s).get_space_group_symbol()
        self.assertEqual(expected, actual)

    def test_a0_bp_cp(self):
        pr = PerovskiteRotater(angels=[0, 2, 3], antiphases=[
                               False, False, True])
        s = get_abx1x2_rotated_structure(pr, self.elements)
        expected = 'Cmcm'  # 63
        actual = SpacegroupAnalyzer(s).get_space_group_symbol()
        self.assertEqual(expected, actual)

    def test_am_bm_cm(self):
        pr = PerovskiteRotater(angels=[1, 2, 3], antiphases=[True, True, True])
        s = get_abx1x2_rotated_structure(pr, self.elements)
        expected = 'P-1'  # 2
        actual = SpacegroupAnalyzer(s).get_space_group_symbol()
        self.assertEqual(expected, actual)


unittest.main(argv=[''], verbosity=2, exit=False)


test_a0_a0_a0 (__main__.TestPerovskiteRotater) ... ok
test_a0_bp_cp (__main__.TestPerovskiteRotater) ... ok
test_am_am_am (__main__.TestPerovskiteRotater) ... ok
test_am_bm_cm (__main__.TestPerovskiteRotater) ... ok
test_ap_bp_cp (__main__.TestPerovskiteRotater) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.036s

OK
